In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np 
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
import random
import ast
import math



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/assignment-1-nlp/sample_submission.csv
/kaggle/input/assignment-1-nlp/test_small.csv
/kaggle/input/assignment-1-nlp/train.csv


In [2]:
df = pd.read_csv('/kaggle/input/assignment-1-nlp/train.csv') # loading training data

In [3]:
untag_data = []
tag_data = []
for index, row in tqdm(df.iterrows()):
    # changing data-type of entries from 'str' to 'list'
    untag_data.append(ast.literal_eval(row['untagged_sentence']))
    tag_data.append(ast.literal_eval(row['tagged_sentence'])) 

47340it [00:16, 2818.19it/s]


In [4]:
test_df = pd.read_csv('/kaggle/input/assignment-1-nlp/test_small.csv', index_col=False) # loading test data

In [5]:
test_data = {} 
for index, row in tqdm(test_df.iterrows()):
    test_data[row['id']] = ast.literal_eval(row['untagged_sentence']) # changing data-type of entries from 'str' to 'list'

4000it [00:00, 6822.09it/s]


<h2> Constructed Data Structures: </h2>

<font size = "3">
    
* untag_data (list) : untagged sentences of training set
    
* tag_data (list) : tagged sentences of training set
    
Dictionaries
    
* word_tags : maps (word) to dictionary of (tags) and their frequency, 'FREQ' tag stores sum frequencies of all of them i.e count(word)

* tag_count : maps (tag) to the number of times it has occured
    
* transition_count: maps (prev_tag, tag) to the number of times it has appeared
   
</font>

In [6]:
word_tags = {}
tag_count = defaultdict(int)
transition_count = defaultdict(int)

# function for constructing all the necessary data structures
def store_tags(tag_count, word_tags, transition_count):    
    
    for i in tqdm(range(len(tag_data))):
        sent = tag_data[i]
        prev_tag = 'START'

        for word, tag in sent:
            if word not in word_tags:
                word_tags[word] = defaultdict(int)
            word_tags[word][tag] += 1
            word_tags[word]['FREQ'] += 1
            tag_count[tag] += 1
            next_tag = tag
            transition_count[(prev_tag, next_tag)] += 1
            prev_tag = next_tag

        next_tag = 'END'
        transition_count[(prev_tag, next_tag)] += 1
        
        # insert "OOV" tag
        word_tags["OOV"] = defaultdict(int)
    return
    
store_tags(tag_count, word_tags, transition_count)

100%|██████████| 47340/47340 [00:02<00:00, 21843.37it/s]


<h3> Constructed Maps: </h3>
<font size = "3"> 
    
* state_map : maps (tag) to corresponding state index <br>
    
* invert_state : maps (state index) to corresponding tag
    
* observation_map : maps (unique words) to corresponding word index
    
* invert_observation : maps (word index) to corresponding unique word 
</font>

In [7]:
# Map Constructions

# Global Variables
N = len(tag_count)   # 49 dist tags in training set: # of states of HMM
K = len(word_tags)   # total distinct words in corpus 

state_map = {tag[0]: i for i, tag in enumerate(tag_count.items())}
invert_state = {value: key for key, value in state_map.items()}
observation_map = {obser: i for i, obser in enumerate(word_tags)}
invert_observation = {value: key for key, value in observation_map.items()}

<h2>HMM Implementation:</h2>


In [8]:
# A, B Initialisation
# PI : 'START' -> () transition probabilities
# Laplace smoothing for all

PI = np.zeros(N, dtype=np.float32)
A = np.zeros((N, N), dtype=np.float32)
B = np.zeros((N, K), dtype=np.float32)

def pi_init(PI, transition_count, invert_state, sm):
    total_sent = len(tag_data)
    for i in tqdm(range(N)):
        num = transition_count[('START', invert_state[i])]
        denom = total_sent
        PI[i] = (num + sm)/(denom + sm*N)
    return PI
        
    
def transition_init(A, tag_count, transition_count, invert_state, sm):    # with smoothing 
    for i in tqdm(range(N)):
        denom = tag_count[invert_state[i]]
        for j in range(N):
            num = transition_count[(invert_state[i], invert_state[j])]
            A[i][j] = (num + sm)/(denom + sm*N)
    return A

# bj(vk) : probability of an observation vk being generated from a state qj
def emission_init(B, KAY, word_tags, invert_state, invert_observation, tag_count, sm):
    for i in tqdm(range(N)):
        denom = tag_count[invert_state[i]]
        for k in range(K):
            if invert_observation[k] == "OOV":
                B[i][k] = KAY[i]
            else:
                num = word_tags[invert_observation[k]][invert_state[i]]
                B[i][k] = (num + sm)/(denom + sm*K)
    return B

def k_bag(word_tags, state_map, invert_state, sm):
    dict_k = defaultdict(int)
    tot_k = 0
    for word, tagdict in tqdm(word_tags.items()):
        if (tagdict['FREQ']) <= 5:   # K = 5
            for key, freq in tagdict.items():
                if key != 'FREQ' and freq != 0:
                    tot_k += 1
                    if key not in dict_k:
                        dict_k[key] = 1
                    else:
                        dict_k[key] += freq
                        
    kay = np.zeros(N, dtype=np.float32)
    for i in range(N):
        tag = invert_state[i]
        kay[i] = (dict_k[tag] + sm)/(tot_k + sm*N)
    return kay
    
# Tune Hyperparameters
KAY = k_bag(word_tags, state_map, invert_state, 0.001)
A = transition_init(A, tag_count, transition_count, invert_state, 0.001)
B = emission_init(B, KAY, word_tags, invert_state, invert_observation, tag_count, 0.00001)
PI = pi_init(PI, transition_count, invert_state, 0.001)

100%|██████████| 49/49 [00:00<00:00, 200900.19it/s]


<h4>Print A, B</h4>

In [9]:
start_row, end_row = 0, 48
start_col, end_col = 0, 10

# Create lists for rows and columns based on the given range
rows = [invert_state[i] for i in range(start_row, end_row + 1)]
cols = [invert_state[j] for j in range(start_col, end_col + 1)]

# Ensure that the lengths of rows and cols match the shape of the A matrix
assert len(rows) == end_row - start_row + 1, "Mismatch in the number of rows"
assert len(cols) == end_col - start_col + 1, "Mismatch in the number of columns"

# Create a DataFrame with the specified rows and columns from the subset of the A matrix
a_sub = pd.DataFrame(A[start_row:end_row + 1, start_col:end_col + 1], index=rows, columns=cols)

# Print the subset DataFrame
# print(a_sub)

In [10]:
start_row, end_row = 0, 48
start_col, end_col = 0, 4

rows = [invert_state[i] for i in range(start_row, end_row + 1)]
cols = [invert_observation[j] for j in range(start_col, end_col + 1)]
    
b_sub = pd.DataFrame(B[start_row:end_row + 1, start_col:end_col + 1], index=rows, columns=cols)
# print(b_sub)

<h3> HMM Training </h3>

<h4> Baum Welch Algorithm </h4>

In [11]:
# Computes alpha, beta matrix via forward backward algorithm

def for_back(sentence, T, A, B, PI):
    # Initialize alpha and beta matrices
    alpha = np.zeros((N, T))
    beta = np.zeros((N, T))

    # Alphas - Forward pass
    observation_prob = B[:, observation_map[sentence[0]]]  # Precompute observation probabilities
    alpha[:, 0] = PI * observation_prob 

    for t in range(1, T):
        observation_prob = B[:, observation_map[sentence[t]]]  # Precompute observation probabilities
        alpha[:, t] = np.dot(alpha[:, t-1], A) * observation_prob

    # Betas - Backward pass
    beta[:, T-1] = 1
        
    # Betas - Backward pass
    for t in range(T-2, -1, -1):
        observation_prob = B[:, observation_map[sentence[t+1]]]  # Precompute observation probabilities
        beta[:, t] = np.dot(A, observation_prob * beta[:, t+1])
    
    # Alpha and Beta termination
    observation_alpha = np.sum(alpha[:, T-1])
    observation_beta = np.sum(beta[:, 0] * B[:, observation_map[sentence[0]]] * PI)

    assert np.isclose(observation_alpha, observation_beta)  # Check for equality
    obs_prob = observation_alpha

    return alpha, beta, obs_prob

In [12]:
# Expectation Step

def expectation(sentence, T, alpha, beta, obs_prob):
    gamma = np.zeros((N, T))
    eta = np.zeros((N, N, T-1))

    # gamma
    gamma[:, :] = alpha * beta / obs_prob

    # eta
    for i in range(N):
        for j in range(N):
            word_next = np.array([observation_map[word] for word in sentence[1:]], dtype = int)  # Convert list to NumPy array
            eta[i, j, :] = (alpha[i, :-1] * A[i, j] * B[j, word_next] * beta[j, 1:]) / obs_prob

    return gamma, eta

In [13]:
# Maximisation Step

def maximisation(T, sentence, gamma, eta, PI):
    
    new_A = np.zeros((N, N))
    new_B = np.zeros((N, K))
    
    new_PI = np.zeros(N)
    
    # new_PI
    new_PI = gamma[:, 0]
    
    # Convert the list of words to an array of indices using observation_map
    word_indices = np.array([observation_map[word] for word in sentence], dtype = int)
    
    # new_A
    for i in range(N):
        for j in range(N):
            
            num = np.sum(eta[i, j, :])  # Sum along the third axis (dimension T-1)
            denom = np.sum(eta[i, :, :])  # Sum along the second axis (dimension N)
            
            if denom == 0:
                new_A[i, j] = 0
            else:
                new_A[i, j] = num / denom
    
    # new_B
    for j in range(N):
        for k in range(K):
            
            num = np.sum(gamma[j, (word_indices == k)])
            denom = np.sum(gamma[j, :])
            
            if denom == 0:
                new_B[j, k] = 0
            else:
                new_B[j, k] = num / denom
    
    return new_A, new_B, new_PI

In [14]:
# HMM Learning Wrapper Function

def hmm_bw(sentence, A, B, PI):
    T = len(sentence)
    alpha, beta, prob = for_back(sentence, T, A, B, PI)
    gamma, eta = expectation(sentence, T, alpha, beta, prob)
    A, B, PI = maximisation(T, sentence, gamma, eta, PI)
    return A, B, PI

<h4> HMM Decode </h4>

In [15]:
def viterbi(untagged_sentence, AV, BV, PIV):
    
    T = len(untagged_sentence)   # length of untagged sentence
    
    vit = np.zeros((N, T), dtype = np.float32) # N states
    backpointer = np.zeros((N, T), dtype = int)
    
    epsilon = 1e-18
    
    # Calculate the initial probabilities outside the loop
    log_pi = np.log(PIV + epsilon)
    
    # Initialisation
    word_zero = untagged_sentence[0]
    for j in range(N):
        vit[j, 0] = log_pi[j] + np.log(BV[j, observation_map[word_zero]] + epsilon)
        backpointer[j, 0] = -1
    
    # Recurrence
    for t in range(1, T):
        word_t = untagged_sentence[t]

        for j in range(N):                
            probs = vit[:, t-1] + np.log(AV[:, j] + epsilon) + math.log(BV[j, observation_map[word_t]] + epsilon)
            bestprob = np.argmax(probs)
            
            backpointer[j][t] = bestprob
            vit[j][t] = probs[bestprob]
        
    # Backtrack v1   
    bestpathpointer = np.argmax(vit[:, T-1])
    tags = []
    for t in range(T-1, -1, -1):
        tags.append((untagged_sentence[t], invert_state[bestpathpointer]))
        bestpathpointer = backpointer[bestpathpointer][t]
    
    return tags[::-1]

#     Print Viterbi Matrix
#     df_vit = pd.DataFrame(vit, columns=[untagged_sentence[i] for i in range(T)], index=[invert_state[i] for i in range(N)])
#     print("Viterbi Matrix (vit):")
#     print(df_vit)

<h4> HMM Tagger </h4>

In [16]:
def hmm_tagger_util(sent_id, untagged_sentence): 
    oov_dict = {}
    
    for i in range(len(untagged_sentence)):
        if untagged_sentence[i] not in observation_map:
            oov_dict[i] = untagged_sentence[i]
            untagged_sentence[i] = "OOV"
    
    # NO OOV CASE - direct decode
    if len(oov_dict) == 0:
        tagged_sentence = viterbi(untagged_sentence, A, B, PI)
        store_submission(sent_id, tagged_sentence)
        return
            
    # MORE THAN ONE OOV CASE - direct decode
    if len(oov_dict) > 1:
        tagged_sentence = viterbi(untagged_sentence, A, B, PI)
        
        # Replacing former values
        for idx, word in oov_dict.items():
            tag = tagged_sentence[idx][1]
            tagged_sentence[idx] = (word, tag)
        
        store_submission(sent_id, tagged_sentence)
        return
    
    # ONE OOV CASE - decode followed by learning for ~ 40% cases
    else:
        
        probability_distribution = [0.4, 0.6] # YES, NO
        learn = random.choices(["yes", "no"], weights=probability_distribution, k=1)[0]
        
        if(learn == "no"):
            tagged_sentence = viterbi(untagged_sentence, A, B, PI)
        
            # Replacing former values
            for idx, word in oov_dict.items():
                tag = tagged_sentence[idx][1]
                tagged_sentence[idx] = (word, tag)

            store_submission(sent_id, tagged_sentence)
            return
        
        # EM LEARNING!
        else:
            AV = np.zeros(A.shape)
            BV = np.zeros(B.shape)
            PIV = np.zeros(PI.shape)
            
            AV, BV, PIV = hmm_bw(untagged_sentence, A, B, PI)
            tagged_sentence = viterbi(untagged_sentence, AV, BV, PIV)
            store_submission(sent_id, tagged_sentence)
            return

<h4> HMM Accuracy Prediction on Train Set: </h4>


In [17]:
def compare_decode(index):
    t = len(untag_data[index])
    hmm_tag = viterbi(untag_data[index])
    
    miss = 0
    for i in range(t):
        if tag_data[index][i][1] != hmm_tag[i][1]:
            miss += 1
            print([tag_data[index][i][0], tag_data[index][i][1], hmm_tag[i][1]])
    return miss

In [18]:
# wrong = 0
# num = 0
# s = 0
# acc = 0
# for sent_idx in tqdm(range(3030,3060)):
#     num = len(untag_data[sent_idx])
#     wrong = compare_decode(sent_idx)
#     acc += ((num - wrong)/(num))*100
#     s += 1
    
# train_accuracy = acc/s
# print(train_accuracy)

<h2>MEMM Implementation:</h2>

In [19]:
import numpy as np

class MEMM:
    
    def __init__(self):
        self.data = []
        self.state_map = {}
        self.invert_state = {}
        self.observation_map = {}
        self.invert_observation = {}
        self.N = 0
        self.K = 0
        self.alpha = 0.01
        self.logP = np.empty((1,1))
        
    def load(self, data):
        self.data = data
        
    def load_dictionaries(self, state_map, invert_state, observation_map, invert_observation):
        self.state_map = state_map
        self.invert_state = invert_state
        self.observation_map = observation_map
        self.invert_observation = invert_observation
        self.N = len(state_map)
        self.K = len(observation_map)
        
    def memm_main(self):
        
        word_features = np.zeros((self.N + 2, self.K))
        
        for sent in self.data:
            prev_tag = 'START'
            for i in range(len(sent)):
                if i == 0:
                    word_features[0, self.observation_map[sent[i][0]]] += 1
                else:
                    word_features[self.state_map[prev_tag], self.observation_map[sent[i][0]]] += 1
                if i == len(sent) - 1:
                    word_features[self.N + 1, self.observation_map[sent[i][0]]] += 1
                prev_tag = sent[i][1]
        
        w = np.random.rand(self.N, self.N + 2)
    
        for epoch in range(5):
            log_p = np.zeros((self.N, self.K))

            for word_ind in range(self.K):
                denom = 0
                for state_ind in range(self.N):
                    log_p[state_ind, word_ind] = np.dot(w[state_ind, :] , word_features[:, word_ind])  # Corrected
                    denom += np.exp(np.dot(w[state_ind, :] , word_features[:, word_ind]))  # Corrected
                log_p[:, word_ind] = log_p[:, word_ind] - np.log(denom)  # Corrected

            grad = np.zeros((self.N, self.N + 2))

            for word in range(self.K):
                for i in range(self.N):
                    for j in range(self.N + 2):
                        grad[i, j] += -(1 - np.exp(log_p[i, word])) * word_features[j, word]  # Corrected

            grad = self.alpha * grad

            w = w - grad
        
        self.logP = np.zeros((self.N, self.K))
        
        for word_ind in range(self.K):
            denom = 0
            for state_ind in range(self.N):
                self.logP[state_ind, word_ind] = np.dot(w[state_ind, :] , word_features[:, word_ind])  # Corrected
                denom += np.exp(np.dot(w[state_ind, :] , word_features[:, word_ind]))  # Corrected
            self.logP[:, word_ind] = self.logP[:, word_ind] - np.log(denom)  # Corrected
    
    def evaluate(self, untagged_sentence):
        
        T = len(untagged_sentence)
        
        vit = np.zeros((self.N, T), dtype=np.float32)  # N states
        backpointer = np.zeros((self.N, T), dtype=int)
        
        epsilon = 1e-18

        # Initialization
        word_zero = untagged_sentence[0]
        for j in range(self.N):  # Corrected
            vit[j, 0] = self.logP[j, self.observation_map[word_zero]]
            backpointer[j, 0] = -1

        # Recurrence
        for t in range(1, T):
            word_t = untagged_sentence[t]

            for j in range(self.N):  # Corrected
                probs = vit[:, t-1] + self.logP[j, self.observation_map[word_t]]
                bestprob = np.argmax(probs)

                backpointer[j, t] = bestprob
                vit[j, t] = probs[bestprob]

        # Backtrack v1   
        bestpathpointer = np.argmax(vit[:, T-1])
        tags = []
        for t in range(T-1, -1, -1):
            tags.append((untagged_sentence[t], self.invert_state[bestpathpointer]))  # Corrected
            bestpathpointer = backpointer[bestpathpointer, t]

        tagged_sentence = tags[::-1]  # Reversed the tags list to get the correct order
        return tagged_sentence

<h4> MEMM Tagger </h4>

In [ ]:
memm = MEMM()
memm.load(tag_data)
memm.load_dictionaries(state_map, invert_state, observation_map, invert_observation)
memm.memm_main()
def memm_tagger_util(sent_id, untagged_sentence):
    tagged_sentence = memm.evaluate(untagged_sentence)
    store_submission(sent_id, tagged_sentence)

<h3>Random Tagger:</h3>

In [43]:
# cell to implement tagger that allots random tags to words in a sentence

def random_tagger_util(sent_id, untagged_sentence):
    if(sent_id in list(submission['id'])):
        return
    tagged_sentence = []
    for word in untagged_sentence:
        tagged_sentence.append((word, random.choice(distinct_tags)))
    store_submission(sent_id, tagged_sentence)
    

<h2>Submission:</h2>

In [44]:
submission = {'id': [], 'tagged_sentence' : []} # dictionary to store tag predictions
# NOTE ---> ensure that tagged_sentence's corresponing 'id' is same as 'id' of corresponding 'untagged_sentence' in training data
def store_submission(sent_id, tagged_sentence):
    
    global submission
    submission['id'].append(sent_id)
    submission['tagged_sentence'].append(tagged_sentence)
    
def clear_submission():
    global submission
    submission = {'id': [], 'tagged_sentence' : []}

<h4>Investigating Unknown Words in Test Set</h4>

In [46]:
# Investigating Unknown Words in Test Set

# count = 0
# tot = 0
# dict_t = {}
# for sent_id in tqdm(list(test_data.keys())):
#     sent = test_data[sent_id]
#     s = 0
#     for word in sent:
#         tot += 1
#         if word_tags.get(word) == None:
#             s += 1
#             count += 1
#             if dict_t.get(word) == None:
#                 dict_t[word] = 1
#             else:
#                 dict_t[word] += 1
#     if s == 1:
#         print(sent_id)
#         break

# print(count/tot)
# print(max(dict_t.values()))
# print(min(dict_t.values()))

In [47]:
# AV = np.zeros(A.shape)
# BV = np.zeros(B.shape)
# PIV = np.zeros(PI.shape)

# for epoch in tqdm(range(1)):
    
#     untagged_sentence = test_data[15]

#     oov_dict = {}

#     for i in range(len(untagged_sentence)):
#         if untagged_sentence[i] not in observation_map:
#             oov_dict[i] = untagged_sentence[i]
#             untagged_sentence[i] = "OOV"

#     AV, BV, PIV = hmm_bw(untagged_sentence, A, B, PI)
#     tagged_sentence = viterbi(untagged_sentence, AV, BV, PIV)

#     # Replacing former values
#     for idx, word in oov_dict.items():
#         tag = tagged_sentence[idx][1]
#         tagged_sentence[idx] = (word, tag)

#     print(tagged_sentence)

In [ ]:
for sent_id in tqdm(list(test_data.keys())):
    sent = test_data[sent_id]
    hmm_tagger_util(sent_id, sent)

In [ ]:
path_to_directory = '/kaggle/working/'
pd.DataFrame(submission).to_csv(path_to_directory +' sample_submission.csv', index = False)

In [ ]:
ans=pd.read_csv("/kaggle/input/assignment-1-nlp/sample_submission.csv")

In [ ]:
ans